## **MONOS CON HABILIDADES - CARLOS MERCADAL**

- Número de Simulaciones realizadas: 100M
- Tiempo de ejecución: 22h 20 minutos

**1) ¿Cómo resolvemos el problema de la homogeneización y limpieza de los datos?. ¿Con cuántos activos nos quedamos?**

Para resolver el problema de la homogeneización de los datos he seguido el mismo enfoque que el seguido para el ejercicio de markowitz con los 70.000 fondos. He generado una función homogeneiazdora que se encarga de filtrar los datos por ISIN y Fechas, creando un único data-frame con todos los fondos y todas las fechas. Posteriormente la función va recorriendo todas las fechas y rellenando los datos para cada fondo. 

Una vez tenemos el data-frame con todos los datos, a esa misma función homogeneizadora le paso un filtro como parámetro para que me deje sólo aquellos fondos que tengan como mínimo el 85% de los días cotizados, y posteriormente le paso la función limpiadora que se encargará de hacer un forward y backward fill para los pocos datos que falten a los fondos. 

En total nos quedamos con alrededor de 17.000 fondos.

**2)¿Cuántas simulaciones vamos a realizar?**

Vamos a realizar 100M de simulaciones

**3) ¿Cómo generamos los números aleatorios y cuántos generamos?¿Cuántos activos seleccionamos para cada simulación?¿Cómo asignamos pesos?**

El mecanismo para la generación de carteras tanto en Alpha de Jensen, Markowitz, Sharpe, y aleatorio, ha sido el mismo. Para cada simulación he generado carteras aleatorias de tamaño 2 a 30, de forma que evitaramos tener el famoso problema de los pesos que teníamos con runif. Una vez hemos seleccionado un tamaño aleatorio, escogemos números aleatorios entre 1 y 100 para el número de activos la cartera, y despues lo dividimos por el sumatorio de todos los números aleatorios. De esde modo, ya tenemos unos pesos aleatorios asignados. Finalmente, sólo queda seleccionar los activos que van a formar parte de la cartera. Para ello, cogemos el tamaño de la cartera (i.e 7) y sacamos 7 fondos aleatorios de nuestro maestro de valores. 

**4) Enfoque de las habilidades de los monos**

En cuanto a las habilidades de los monos, primero he calculado el alpha de jensen y el ratio de sharpe para ventanas de 15 a 60 días de 5 en 5. De este modo, los monos que tengan asignadas las habilidades de alpha o de sharpe, no sólo van a escoger carteras aleatorias basados en los alphas o sharpes de una ventana concreta, si no que enriquecemos la simulacion dando la posibilidad de que cada mono tenga unas ventanas distintas. 

Para Markowitz simplemente me he limitado a seguir las indicaciones del ejercicio, escogiendo una ventana de 250 días (1 año, aproximadamente). En cuanto a las simulaciones de markowitz dentro de cada mono, he obtado por hacer únicamente 100 carteras de pesos aleatorios. La razón de hacer esa cantidad se basa en que el mero hecho de generar miles de pesos no va a conllevar un resultado muy distinto y eficientemente es más rapido. Además, no debemos olvidar que estamos haciendo millones de simulaciones de monos para una misma ventana de markowitz, por lo que realmente no estamos "perdiendo" aleatoriedad, si no que ésta sigue existiendo y ganamos más velocidad. 

**5) Reinversión de los monos**

La reinversión de los monos la he solventado con un bucle "while", indicando que para cada mono si su rentabilidad no es inferior al umbral establecido en el enunciado, pueda reinvertir hasta que se quede sin "días" para poder invertir. 

In [2]:
import numpy as np
import pandas as pd
import funciones_monos
import pickle
from tqdm import tqdm
import itertools
import multiprocessing as mp
import time
import matplotlib.pyplot as plt
from datetime import datetime
import random

#### **Cargamos los datos proporcionados y los homogeneizamos**

In [ ]:
navs = pickle.load(open(r'C:\Users\Carlos\Documents\MIA-X\Modulo 2\Ejercicios Voluntarios\Monos con habilidades\navs.pickle','rb'))
df_allfunds = pd.concat(navs)
df_allfunds = df_allfunds.droplevel(level=0, axis=0)

In [ ]:
porcentaje_dias_cotizados = 0.85
fondos_homogeneizados = funciones_monos.homogeneización_fondos(df_allfunds, porcentaje_dias_cotizados)

In [2]:
datos_fondos = pickle.load(open(r'C:\Users\Carlos\Documents\MIA-X\Modulo 2\Ejercicios Voluntarios\Monos con habilidades\fondos_homogeneizados.pkl','rb'))
datos_fondos.columns = datos_fondos.columns.map(''.join)
rentabilidad_fondos = np.array(np.log(pd.DataFrame(datos_fondos)).diff().dropna())

In [3]:
maestro_fondos = pd.read_csv(r"C:\Users\Carlos\Documents\MIA-X\Modulo 2\Ejercicios Voluntarios\Monos con habilidades\maestro.csv", encoding="utf-8", parse_dates=True, index_col=0)

In [4]:
maestro_fondos.name[maestro_fondos.name.str.contains("AMUNDI INDEX MSCI WORLD")] #Identificamos posibles Benchmark, y nos quedamos con el que más nos guste

10512    AMUNDI INDEX MSCI WORLD "IU" (USD) INC
10513    AMUNDI INDEX MSCI WORLD "AU" (USD) INC
10514    AMUNDI INDEX MSCI WORLD "RE" (EUR) ACC
10515    AMUNDI INDEX MSCI WORLD "IE-EXF" (EUR)
10516    AMUNDI INDEX MSCI WORLD "AE" (EUR) ACC
10518    AMUNDI INDEX MSCI WORLD "AE" (EUR) INC
10519    AMUNDI INDEX MSCI WORLD "IE" (EUR) ACC
10520    AMUNDI INDEX MSCI WORLD "AU" (USD) ACC
10521    AMUNDI INDEX MSCI WORLD "IE" (EUR) INC
Name: name, dtype: object

In [5]:
isin_benchmark = maestro_fondos.iloc[10516]["isin"] #Selecciono el de la posición 10516
msci_world = datos_fondos.loc[:,isin_benchmark] #Lo guardamos en otra variable 
datos_fondos = datos_fondos.drop(isin_benchmark, axis = 1) #se elimina del data-frame
datos_fondos = np.array(datos_fondos)

**Generamos las ventanas para el Alpha Jensen**

In [ ]:
start_time = time.time()
ventana = np.arange(15,65,5)
lista_ventana = ventana.tolist()
lista_fondos = [datos_fondos]
lista_indice = [msci_world]

all_combinations = itertools.product(lista_fondos, lista_indice, lista_ventana)
pool = mp.Pool(mp.cpu_count()-2)
if __name__ == "__main__":
    alphas = pool.starmap(funciones_monos.alpha_jensen, all_combinations)
ventanas_alpha = {}
count = 0
for ventana in lista_ventana:
    ventanas_alpha[ventana] = np.array(alphas[lista_ventana.index(ventana)][0])
pickle.dump(ventanas_alpha, open("ventanas_alpha.pickle", "wb"))
print("--- %s seconds ---" % (time.time() - start_time))

In [6]:
ventanas_alpha = pickle.load(open(r'C:\Users\Carlos\Documents\MIA-X\Modulo 2\Ejercicios Voluntarios\Monos con habilidades\ventanas_alpha.pickle','rb'))

**Generamos las ventanas para el Ratio Sharpe**

In [ ]:
ventana = np.arange(15,65,5)
sharpe_ratio = funciones_monos.ratio_sharpe(datos_fondos, ventana)
pickle.dump(sharpe_ratio, open("sharpe_ratio.pickle", "wb"))

In [7]:
ventanas_sharpe = pickle.load(open(r'C:\Users\Carlos\Documents\MIA-X\Modulo 2\Ejercicios Voluntarios\Monos con habilidades\sharpe_ratio.pickle','rb'))

**GENERACIÓN DE MONOS Y ASIGNACIÓN DE HABILIDADES**

Primera tanda

In [8]:
start_time = time.time()
n_simulaciones = 50000000
porcentajes_habilidades = np.random.randint(1,100, size = 4)
porcentajes_habilidades = porcentajes_habilidades/np.sum(porcentajes_habilidades)
simulaciones_alpha = np.round(porcentajes_habilidades[0] * n_simulaciones)
simulaciones_sharpe = np.round(porcentajes_habilidades[1] * n_simulaciones)
simulaciones_marko = np.round(porcentajes_habilidades[2] * n_simulaciones)
simulaciones_aleatorio = np.round(porcentajes_habilidades[3] * n_simulaciones)
print('ALPHA:', simulaciones_alpha)
print('SHARPE:', simulaciones_sharpe)
print('MARKOWITZ:', simulaciones_marko)
print('ALEATORIO:', simulaciones_aleatorio)

ALPHA: 5978261.0
SHARPE: 4891304.0
MARKOWITZ: 28804348.0
ALEATORIO: 10326087.0


Segunda tanda

In [13]:
n_simulaciones = 50000000
porcentajes_habilidades = np.random.randint(1,100, size = 4)
porcentajes_habilidades = porcentajes_habilidades/np.sum(porcentajes_habilidades)
simulaciones_alpha = np.round(porcentajes_habilidades[0] * n_simulaciones)
simulaciones_sharpe = np.round(porcentajes_habilidades[1] * n_simulaciones)
simulaciones_marko = np.round(porcentajes_habilidades[2] * n_simulaciones)
simulaciones_aleatorio = np.round(porcentajes_habilidades[3] * n_simulaciones)
print('ALPHA:', simulaciones_alpha)
print('SHARPE:', simulaciones_sharpe)
print('MARKOWITZ:', simulaciones_marko)
print('ALEATORIO:', simulaciones_aleatorio)

ALPHA: 8739837.0
SHARPE: 3048780.0
MARKOWITZ: 18699187.0
ALEATORIO: 19512195.0


**Calculamos el Alpha**

In [10]:
#Calculamos el Alpha
rentabilidad_monos_alpha = []
for repeticiones in tqdm(range(simulaciones_alpha.astype(int))):
    ventana = np.random.choice(np.arange(15,65,5))
    rentabilidad_alpha = funciones_monos.habilidad_alpha(ventanas_alpha, datos_fondos, ventana)
    rentabilidad_monos_alpha.append( rentabilidad_alpha)
pickle.dump(rentabilidad_monos_alpha, open("alpha_rentabilidad_monos.pkl", "wb"))

100%|██████████| 5978261/5978261 [3:18:10<00:00, 502.76it/s]  


In [8]:
#Calculamos el Alpha
rentabilidad_monos_alpha = []
for repeticiones in  tqdm(range(int(simulaciones_alpha))):
    ventana = np.random.choice(np.arange(15,65,5))
    rentabilidad_alpha = funciones_monos.habilidad_alpha(ventanas_alpha, datos_fondos, ventana)
    rentabilidad_monos_alpha.append( rentabilidad_alpha)
pickle.dump(rentabilidad_monos_alpha, open("alpha_segunda_tanda.pkl", "wb"))

100%|██████████| 8739837/8739837 [4:28:48<00:00, 541.88it/s]  


**Calculamos el Sharpe**

In [11]:
#Calculamos Sharpe
rentabilidad_monos_sharpe = []
for repeticiones in tqdm(range(simulaciones_sharpe.astype(int))):
    ventana = np.random.choice(np.arange(15,65,5))
    rentabilidad_sharpe = funciones_monos.habilidad_sharpe(ventanas_sharpe, datos_fondos, ventana)
    rentabilidad_monos_sharpe.append(rentabilidad_sharpe)
pickle.dump(rentabilidad_monos_sharpe, open("rentabilidad_monos_sharpe.pkl", "wb"))

100%|██████████| 4891304/4891304 [2:31:57<00:00, 536.46it/s]  


In [19]:
#Calculamos Sharpe
rentabilidad_monos_sharpe = []
for repeticiones in tqdm(range(simulaciones_sharpe.astype(int))):
    ventana = np.random.choice(np.arange(15,65,5))
    rentabilidad_sharpe = funciones_monos.habilidad_sharpe(ventanas_sharpe, datos_fondos, ventana)
    rentabilidad_monos_sharpe.append(rentabilidad_sharpe)
pickle.dump(rentabilidad_monos_sharpe, open("sharpe_segunda_tanda.pkl", "wb"))

100%|██████████| 3048780/3048780 [1:35:03<00:00, 534.57it/s]


**Calculamos Markowitz**

In [7]:
#Calculamos Markowitz
lista_datos = [datos_fondos]
lista_rentabilidades = [rentabilidad_fondos]
rentabilidad_monos_marko = []
                                    
for repeticiones in tqdm(range(4)):  
    all_combinations = itertools.product(lista_datos, lista_rentabilidades, range(int(simulaciones_marko/4)))    
    pool = mp.Pool(mp.cpu_count()-2)
    if __name__ == "__main__":
        rentabilidad_marko = pool.starmap(funciones_monos.habilidad_markowitz, all_combinations)
        rentabilidad_monos_marko.append(rentabilidad_marko)
        pickle.dump(rentabilidad_monos_marko, open("rentabilidad_monos_marko.pkl", "wb"))

100%|██████████| 4/4 [5:28:25<00:00, 4926.26s/it]  


In [16]:
#Calculamos Markowitz
lista_datos = [datos_fondos]
lista_rentabilidades = [rentabilidad_fondos]
rentabilidad_monos_marko = []
                                    
for repeticiones in tqdm(range(13)):  
    all_combinations = itertools.product(lista_datos, lista_rentabilidades, range(int(simulaciones_marko/13)))    
    pool = mp.Pool(mp.cpu_count()-2)
    if __name__ == "__main__":
        rentabilidad_marko = pool.starmap(funciones_monos.habilidad_markowitz, all_combinations)
        rentabilidad_monos_marko.append(rentabilidad_marko)
        pickle.dump(rentabilidad_monos_marko, open("marko_segunda_tanda.pkl", "wb"))

100%|██████████| 13/13 [3:32:20<00:00, 980.05s/it] 


**Calculamos los Aleatorios**

In [12]:
#Calculamos Aleatorios
rentabilidad_monos_aleatorio = []
for repeticiones in tqdm(range(int(simulaciones_aleatorio))):
    rentabilidad_aleatorio = funciones_monos.habilidad_aleatorio(datos_fondos)
    rentabilidad_monos_aleatorio.append(rentabilidad_aleatorio)
pickle.dump(rentabilidad_monos_aleatorio, open("rentabilidad_monos_aleatorio.pkl", "wb"))


100%|██████████| 10326087/10326087 [50:18<00:00, 3420.48it/s]


In [9]:
#Calculamos Aleatorios
rentabilidad_monos_aleatorio = []
for repeticiones in tqdm(range(int(simulaciones_aleatorio))):
    rentabilidad_aleatorio = funciones_monos.habilidad_aleatorio(datos_fondos)
    rentabilidad_monos_aleatorio.append(rentabilidad_aleatorio)
pickle.dump(rentabilidad_monos_aleatorio, open("aleatorios_segunda_tanda.pkl", "wb"))

100%|██████████| 19512195/19512195 [1:33:19<00:00, 3484.66it/s]


**Cargamos los resultados de las simulaciones**

In [48]:
marko_primera = pickle.load(open(r'C:\Users\Carlos\Documents\MIA-X\Modulo 2\Ejercicios Voluntarios\Monos con habilidades\rentabilidad_monos_marko.pkl','rb'))
marko_segunda = pickle.load(open(r'C:\Users\Carlos\Documents\MIA-X\Modulo 2\Ejercicios Voluntarios\Monos con habilidades\marko_segunda_tanda.pkl','rb'))
alpha_primera = pickle.load(open(r'C:\Users\Carlos\Documents\MIA-X\Modulo 2\Ejercicios Voluntarios\Monos con habilidades\alpha_rentabilidad_monos.pkl','rb'))
alpha_segunda = pickle.load(open(r'C:\Users\Carlos\Documents\MIA-X\Modulo 2\Ejercicios Voluntarios\Monos con habilidades\alpha_segunda_tanda.pkl','rb'))
sharpe_primera = pickle.load(open(r'C:\Users\Carlos\Documents\MIA-X\Modulo 2\Ejercicios Voluntarios\Monos con habilidades\rentabilidad_monos_sharpe.pkl','rb'))
sharpe_segunda = pickle.load(open(r'C:\Users\Carlos\Documents\MIA-X\Modulo 2\Ejercicios Voluntarios\Monos con habilidades\sharpe_segunda_tanda.pkl','rb'))
aleatorio_primera = pickle.load(open(r'C:\Users\Carlos\Documents\MIA-X\Modulo 2\Ejercicios Voluntarios\Monos con habilidades\rentabilidad_monos_aleatorio.pkl','rb'))
aleatorios_segunda = pickle.load(open(r'C:\Users\Carlos\Documents\MIA-X\Modulo 2\Ejercicios Voluntarios\Monos con habilidades\aleatorios_segunda_tanda.pkl','rb'))

In [62]:
resultados_markowitz = marko_primera + marko_segunda
resultados_alpha = alpha_primera + alpha_segunda
resultados_sharpe = sharpe_primera + sharpe_segunda
resultados_aleatorio = aleatorio_primera + aleatorios_segunda

In [64]:
resultados_markowitz = ((np.array([item for sublist in resultados_markowitz for item in sublist]))-1)*1000
resultados_alpha = (np.array(resultados_alpha)-1)*1000
resultados_sharpe = (np.array(resultados_sharpe)-1)*1000
resultados_aleatorio = (np.array(resultados_aleatorio)-1)*1000

**Resultados Cuantiles Markowitz**

In [82]:
(pd.Series(resultados_markowitz.round(2))).quantile([0, .1, .2, .3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])

0.0       0.69
0.1      43.98
0.2      52.90
0.3      59.81
0.4      66.07
0.5      71.98
0.6      78.26
0.7      85.17
0.8      93.58
0.9     105.92
1.0    1888.18
dtype: float64

**Resultados Cuantiles Alpha**

In [83]:
(pd.Series(resultados_alpha.round(2))).quantile([0, .1, .2, .3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])

0.0   -9.800900e+02
0.1    2.614670e+03
0.2    3.875180e+03
0.3    5.149610e+03
0.4    6.584840e+03
0.5    8.317740e+03
0.6    1.055978e+04
0.7    1.371466e+04
0.8    1.881763e+04
0.9    2.979781e+04
1.0    2.302360e+08
dtype: float64

**Resultados Cuantiles Sharpe**

In [84]:
(pd.Series(resultados_sharpe.round(2))).quantile([0, .1, .2, .3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])

0.0   -9.822600e+02
0.1    2.708250e+03
0.2    4.030980e+03
0.3    5.376190e+03
0.4    6.895500e+03
0.5    8.738905e+03
0.6    1.112267e+04
0.7    1.448694e+04
0.8    1.993983e+04
0.9    3.168852e+04
1.0    2.683038e+07
dtype: float64

**Resultados Cuantiles Aleatorio**

In [85]:
(pd.Series(resultados_aleatorio.round(2))).quantile([0, .1, .2, .3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])

0.0       -973.64
0.1        243.13
0.2        296.40
0.3        333.15
0.4        364.68
0.5        394.87
0.6        426.32
0.7        462.14
0.8        508.47
0.9        586.75
1.0    1664526.60
dtype: float64